# CENSUS DATA


# EDA

## Data Exploration

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


### Import data

##### Zip Code Data
- using for city information

In [ ]:
zip_to_city  = pd.read_csv('/content/uszips.csv')

In [ ]:
zip_to_city.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18005,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17113.0,102.7,72001,Adjuntas,"{""72001"": ""99.43"", ""72141"": ""0.57""}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36074,-67.17519,Aguada,PR,Puerto Rico,True,NaN,37751.0,476.0,72003,Aguada,"{""72003"": ""100""}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45440,-67.12201,Aguadilla,PR,Puerto Rico,True,NaN,47081.0,574.9,72005,Aguadilla,"{""72005"": ""100""}",Aguadilla,72005,False,False,America/Puerto_Rico
3,606,18.16721,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6392.0,58.3,72093,Maricao,"{""72093"": ""94.88"", ""72153"": ""3.78"", ""72121"": ""...",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29032,-67.12244,Anasco,PR,Puerto Rico,True,NaN,26686.0,286.9,72011,Añasco,"{""72011"": ""99.45"", ""72003"": ""0.55""}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


In [ ]:
zip_to_city = zip_to_city.rename(columns = {'state_name': 'State', 
                                            'state_id': 'State_ID',
                                            'city': 'City', 
                                            'county_name': 'County',
                                            'county_fips': 'County_Fips'
                                        })

In [ ]:
print(zip_to_city['State'].nunique())
zip_to_city['State'].unique()

56


array(['Puerto Rico', 'Virgin Islands', 'Massachusetts', 'Rhode Island',
       'New Hampshire', 'Maine', 'Vermont', 'Connecticut', 'New York',
       'New Jersey', 'Pennsylvania', 'Delaware', 'District of Columbia',
       'Virginia', 'Maryland', 'West Virginia', 'North Carolina',
       'South Carolina', 'Georgia', 'Florida', 'Alabama', 'Tennessee',
       'Mississippi', 'Kentucky', 'Ohio', 'Indiana', 'Michigan', 'Iowa',
       'Wisconsin', 'Minnesota', 'South Dakota', 'North Dakota',
       'Montana', 'Illinois', 'Missouri', 'Kansas', 'Nebraska',
       'Louisiana', 'Arkansas', 'Oklahoma', 'Texas', 'Colorado',
       'Wyoming', 'Idaho', 'Utah', 'Arizona', 'New Mexico', 'Nevada',
       'California', 'Hawaii', 'American Samoa', 'Guam',
       'Northern Mariana Islands', 'Oregon', 'Washington', 'Alaska'],
      dtype=object)

In [ ]:
zip_to_city = zip_to_city[zip_to_city['State'] != 'Virgin Islands']
zip_to_city = zip_to_city[zip_to_city['State'] != 'Puerto Rico']
zip_to_city = zip_to_city[zip_to_city['State'] != 'District of Columbia']
zip_to_city = zip_to_city[zip_to_city['State'] != 'American Samoa']
zip_to_city = zip_to_city[zip_to_city['State'] != 'Guam']
zip_to_city = zip_to_city[zip_to_city['State'] != 'Northern Mariana Islands']

In [ ]:
zip_to_city.head()

,zip,lat,lng,City,State_ID,State,zcta,parent_zcta,population,density,County_Fips,County,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
142,1001,42.06259,-72.62589,Agawam,MA,Massachusetts,True,NaN,17312.0,581.0,25013,Hampden,"{""25013"": ""100""}",Hampden,25013,False,False,America/New_York
143,1002,42.37492,-72.46210,Amherst,MA,Massachusetts,True,NaN,30014.0,210.5,25015,Hampshire,"{""25015"": ""99.03"", ""25011"": ""0.97""}",Hampshire|Franklin,25015|25011,False,False,America/New_York
144,1003,42.39192,-72.52479,Amherst,MA,Massachusetts,True,NaN,11357.0,6164.3,25015,Hampshire,"{""25015"": ""100""}",Hampshire,25015,False,False,America/New_York
145,1005,42.42017,-72.10615,Barre,MA,Massachusetts,True,NaN,5128.0,44.7,25027,Worcester,"{""25027"": ""100""}",Worcester,25027,False,False,America/New_York
146,1007,42.27875,-72.40036,Belchertown,MA,Massachusetts,True,NaN,15005.0,110.1,25015,Hampshire,"{""25015"": ""100""}",Hampshire,25015,False,False,America/New_York


##### Census 
- Contains population, demographic, poverty, and employment data by tract

In [ ]:
df = pd.read_csv('/content/acs2017_census_tract_data.csv')
print(df.shape)
df.head()

(74001, 37)


,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,3.1,0.0,7229,68143.0,14424.0,36905.0,10706.0,12.2,17.9,48.8,13.8,20.5,3.5,13.4,86.9,11.2,0.0,0.8,0.3,0.7,21.0,4787,71.4,24.1,4.5,0.0,2.3


##### Tract to Zip
- maps census tracts to zip codes

In [ ]:
tract_zip = pd.read_excel('/content/TRACT_ZIP_122017.xlsx')

In [ ]:
print(tract_zip.shape)
tract_zip.head()

(168662, 6)


,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1001020100,36067,1.000000,1.000000,1.00,1.000000
1,1001020200,36008,0.028243,0.012987,0.00,0.023846
2,1001020200,36067,0.971757,0.987013,1.00,0.976154
3,1001020300,36067,1.000000,1.000000,1.00,1.000000
4,1001020400,36067,0.025543,0.585987,0.25,0.071038


In [ ]:
tract_zip = tract_zip[['tract', 'zip', 'res_ratio']]

In [ ]:
tract_zip = tract_zip.rename(columns = {'tract':'TractId'})

In [ ]:
tract_zip['zip'].nunique()

39301

## Merge Census with Tract_Zip Data

In [ ]:
merge = df.merge(tract_zip, on=['TractId'])

In [ ]:
print(merge.shape)
merge.head()

(168385, 39)


,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6,36067,1.000000
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4,36008,0.028243
2,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4,36067,0.971757
3,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7,36067,1.000000
4,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1,36067,0.025543


### Seeing what does not transfer over

In [ ]:
df[(~df.TractId.isin(merge.TractId))]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
43,1003990000,Alabama,Baldwin County,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
868,1097990000,Alabama,Mobile County,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1266,2105000200,Alaska,Hoonah-Angoon Census Area,62,35,27,0.0,11.3,0.0,82.3,0.0,0.0,60,42500.0,9004.0,23827.0,3232.0,6.5,50.0,39.3,32.1,7.1,10.7,10.7,51.9,0.0,0.0,25.9,0.0,22.2,13.8,28,32.1,35.7,32.1,0.0,12.5
1324,2185000100,Alaska,North Slope Borough,4444,2366,2078,5.3,13.3,0.3,59.4,10.5,3.3,2769,82964.0,13376.0,31860.0,5094.0,11.2,13.0,35.6,15.0,26.4,7.4,15.6,48.1,26.8,0.5,11.9,9.0,3.8,6.6,1800,44.3,54.4,1.3,0.0,14.2
1353,4001944202,Arizona,Apache County,4094,1933,2161,3.4,0.5,0.4,93.9,0.2,0.0,2561,26296.0,6947.0,10079.0,1702.0,53.9,70.0,43.8,17.6,12.8,13.4,12.3,73.1,13.7,2.1,7.5,0.0,3.5,22.8,851,39.4,56.9,3.8,0.0,28.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73982,72151950700,Puerto Rico,Yabucoa Municipio,4744,2377,2367,100.0,0.0,0.0,0.0,0.0,0.0,4061,16087.0,3373.0,8157.0,1248.0,53.6,42.2,41.0,12.1,22.1,12.9,11.9,85.5,14.5,0.0,0.0,0.0,0.0,33.9,1034,61.4,36.2,2.4,0.0,17.9
73983,72151950800,Puerto Rico,Yabucoa Municipio,3531,1915,1616,100.0,0.0,0.0,0.0,0.0,0.0,2924,15767.0,2603.0,8133.0,1350.0,42.4,52.7,12.7,31.9,21.2,9.2,25.0,81.1,13.7,0.0,2.7,2.5,0.0,25.0,780,71.0,18.2,10.8,0.0,30.4
73984,72151950900,Puerto Rico,Yabucoa Municipio,5822,2892,2930,99.7,0.3,0.0,0.0,0.0,0.0,4380,13841.0,3098.0,7088.0,1162.0,58.7,66.5,29.0,24.1,25.5,9.2,12.2,95.7,2.0,0.0,2.3,0.0,0.0,32.3,1143,66.0,29.4,4.6,0.0,28.7
73989,72151990000,Puerto Rico,Yabucoa Municipio,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
tract_zip[(~tract_zip.TractId.isin(merge.TractId))]

,TractId,zip,res_ratio
543,1037030200,36080,1.0
2828,1115050101,35146,1.0
3017,1123231500,36866,1.0
3673,2270000100,99563,0.0
3674,2270000100,99620,0.0
...,...,...,...
168657,78030960800,802,1.0
168658,78030960900,802,1.0
168659,78030961000,802,1.0
168660,78030961100,802,1.0


## Data Cleaning

### isnull().sum()

In [ ]:
merge.isnull().sum()

TractId                0
State                  0
County                 0
TotalPop               0
Men                    0
Women                  0
Hispanic             506
White                506
Black                506
Native               506
Asian                506
Pacific              506
VotingAgeCitizen       0
Income              1291
IncomeErr           1291
IncomePerCap         596
IncomePerCapErr      596
Poverty              710
ChildPoverty        1286
Professional         652
Service              652
Office               652
Construction         652
Production           652
Drive                633
Carpool              633
Transit              633
Walk                 633
OtherTransp          633
WorkAtHome           633
MeanCommute          949
Employed               0
PrivateWork          652
PublicWork           652
SelfEmployed         652
FamilyWork           652
Unemployment         650
zip                    0
res_ratio              0
dtype: int64

### Fill na with 0

In [ ]:
merge.fillna(0, inplace=True)

In [ ]:
merge.isnull().sum()

TractId             0
State               0
County              0
TotalPop            0
Men                 0
Women               0
Hispanic            0
White               0
Black               0
Native              0
Asian               0
Pacific             0
VotingAgeCitizen    0
Income              0
IncomeErr           0
IncomePerCap        0
IncomePerCapErr     0
Poverty             0
ChildPoverty        0
Professional        0
Service             0
Office              0
Construction        0
Production          0
Drive               0
Carpool             0
Transit             0
Walk                0
OtherTransp         0
WorkAtHome          0
MeanCommute         0
Employed            0
PrivateWork         0
PublicWork          0
SelfEmployed        0
FamilyWork          0
Unemployment        0
zip                 0
res_ratio           0
dtype: int64

In [ ]:
merge.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

### Remove nonstates

In [ ]:
# Remove areas we don't have in other df
merge = merge[merge['State'] != 'District of Columbia']
merge = merge[merge['State'] != 'Puerto Rico']

In [ ]:
merge['State'].nunique()

50

## Map Tracts to Zip Code
* first get multiply % columns by total population
* multiply residential ratio across rows

### Dividing certain columns by 100


##### Making sure I capture the correct columns

In [ ]:
merge.iloc[:,6:11+1]

,Hispanic,White,Black,Native,Asian,Pacific
0,2.4,86.3,5.2,0.0,1.2,0.0
1,1.1,41.6,54.5,0.0,1.0,0.0
2,1.1,41.6,54.5,0.0,1.0,0.0
3,8.0,61.4,26.5,0.6,0.7,0.4
4,9.6,80.3,7.1,0.5,0.2,0.0
...,...,...,...,...,...,...
166947,3.0,89.4,1.0,0.1,3.0,0.0
166948,3.0,89.4,1.0,0.1,3.0,0.0
166949,3.0,89.4,1.0,0.1,3.0,0.0
166950,3.0,89.4,1.0,0.1,3.0,0.0


In [ ]:
merge.iloc[:,17:36+1]

,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
3,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
4,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166947,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4
166948,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4
166949,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4
166950,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4


In [ ]:
merge.iloc[:,6:11+1] /= 100

In [ ]:
merge.iloc[:,17:36+1] /= 100

In [ ]:
merge.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
0,1001020100,Alabama,Autauga County,1845,899,946,0.024,0.863,0.052,0.000,0.012,0.000,1407,67826.0,14560.0,33018.0,6294.0,0.107,0.208,0.385,0.156,0.228,0.108,0.124,0.942,0.033,0.0,0.005,0.000,0.021,0.245,8.81,0.742,0.212,0.045,0.000,0.046,36067,1.000000
1,1001020200,Alabama,Autauga County,2172,1167,1005,0.011,0.416,0.545,0.000,0.010,0.000,1652,41287.0,3819.0,18996.0,2453.0,0.224,0.358,0.305,0.249,0.229,0.063,0.154,0.905,0.091,0.0,0.000,0.005,0.000,0.222,8.52,0.759,0.150,0.090,0.000,0.034,36008,0.028243
2,1001020200,Alabama,Autauga County,2172,1167,1005,0.011,0.416,0.545,0.000,0.010,0.000,1652,41287.0,3819.0,18996.0,2453.0,0.224,0.358,0.305,0.249,0.229,0.063,0.154,0.905,0.091,0.0,0.000,0.005,0.000,0.222,8.52,0.759,0.150,0.090,0.000,0.034,36067,0.971757
3,1001020300,Alabama,Autauga County,3385,1533,1852,0.080,0.614,0.265,0.006,0.007,0.004,2480,46806.0,9496.0,21236.0,2562.0,0.147,0.211,0.279,0.194,0.333,0.099,0.096,0.883,0.084,0.0,0.010,0.008,0.015,0.231,14.82,0.733,0.211,0.048,0.007,0.047,36067,1.000000
4,1001020400,Alabama,Autauga County,4267,2001,2266,0.096,0.803,0.071,0.005,0.002,0.000,3257,55895.0,4369.0,28068.0,3190.0,0.023,0.017,0.290,0.166,0.258,0.091,0.195,0.823,0.112,0.0,0.015,0.029,0.021,0.259,18.49,0.758,0.197,0.045,0.000,0.061,36067,0.025543


##### Multiply Columns by Total Population

In [ ]:
merge.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'zip', 'res_ratio'],
      dtype='object')

In [ ]:
col = ['Hispanic','White', 'Black', 'Native', 'Asian', 'Pacific','Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment']

In [ ]:
merge[col] = merge[col].multiply(merge['TotalPop'], axis = 'index')

In [ ]:
merge[col] = merge[col].astype(int)

##### Check values

In [ ]:
df.loc[
       (df.State == 'Wyoming')
       &(df.TotalPop == 3329)
       &(df.TractId == 56045951100)]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
73054,56045951100,Wyoming,Weston County,3329,1946,1383,3.0,89.4,1.0,0.1,3.0,0.0,2700,62435.0,9525.0,32665.0,4047.0,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4


In [ ]:
merge.loc[166948:166948]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
166948,56045951100,Wyoming,Weston County,3329,1946,1383,99,2976,33,3,99,0,2700,62435.0,9525.0,32665.0,4047.0,416,615,1071,529,446,682,595,2403,266,316,69,99,173,1041,52698,2150,762,362,53,13,82729,0.002424


In [ ]:
int(3/100 * 3329)

99

In [ ]:
int(89.4/100 * 3329)

2976

In [ ]:
assert merge['Hispanic'][166948] == 99

In [ ]:
assert merge['White'][166948] == 2976

In [ ]:
df.loc[
       (df.State == 'Alabama')
       &(df.TotalPop == 7131)
       &(df.TractId == 1003011601)]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
41,1003011601,Alabama,Baldwin County,7131,3699,3432,9.7,87.6,1.4,0.7,0.1,0.0,5232,49900.0,9397.0,21858.0,3270.0,22.4,35.1,25.4,23.9,22.6,11.3,16.7,83.4,7.4,0.0,0.5,0.3,8.4,29.3,3260,76.8,9.5,13.5,0.2,3.1


In [ ]:
merge.loc[105:105]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
105,1003011601,Alabama,Baldwin County,7131,3699,3432,691,6246,99,49,7,0,5232,49900.0,9397.0,21858.0,3270.0,1597,2502,1811,1704,1611,805,1190,5947,527,0,35,21,599,2089,232470,5476,677,962,14,221,36580,0.01071


In [ ]:
int(25.4/100 * 7131)

1811

In [ ]:
int(11.3/100 * 7131)

805

In [ ]:
assert merge['Professional'][105] == 1811

In [ ]:
assert merge['Construction'][105] == 805

### Mutiply each row by residential ratio

In [ ]:
newcol = ['TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment']

In [ ]:
merge[newcol] = merge[newcol].multiply(merge['res_ratio'], axis = 'index')

In [ ]:
merge[newcol] = merge[newcol].astype(int)

In [ ]:
merge.head(15)

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
0,1001020100,Alabama,Autauga County,1845,899,946,44,1592,95,0,22,0,1407,67826,14560,33018,6294,197,383,710,287,420,199,228,1737,60,0,9,0,38,452,16254,1368,391,83,0,84,36067,1.000000
1,1001020200,Alabama,Autauga County,61,32,28,0,25,33,0,0,0,46,1166,107,536,69,13,21,18,15,14,3,9,55,5,0,0,0,0,13,522,46,9,5,0,2,36008,0.028243
2,1001020200,Alabama,Autauga County,2110,1134,976,22,877,1149,0,20,0,1605,40120,3711,18459,2383,472,755,643,524,482,132,324,1909,191,0,0,9,0,468,17982,1601,315,189,0,70,36067,0.971757
3,1001020300,Alabama,Autauga County,3385,1533,1852,270,2078,897,20,23,13,2480,46806,9496,21236,2562,497,714,944,656,1127,335,324,2988,284,0,33,27,50,781,50165,2481,714,162,23,159,36067,1.000000
4,1001020400,Alabama,Autauga County,108,51,57,10,87,7,0,0,0,83,1427,111,716,81,2,1,31,18,28,9,21,89,12,0,1,3,2,28,2015,82,21,4,0,6,36067,0.025543
5,1001020400,Alabama,Autauga County,4158,1949,2208,398,3338,294,20,7,0,3173,54467,4257,27351,3108,95,70,1205,689,1071,378,810,3421,464,0,62,119,86,1076,76880,3151,818,187,0,253,36066,0.974457
6,1001020500,Alabama,Autauga County,7856,3984,3871,70,6087,1288,0,242,0,5699,53723,11371,29095,8440,957,1405,3833,1084,1609,274,1052,6826,879,0,62,22,54,1649,376081,5609,1892,353,0,180,36066,0.788390
7,1001020500,Alabama,Autauga County,2100,1065,1035,18,1627,344,0,64,0,1523,14364,3040,7779,2256,256,375,1024,289,430,73,281,1825,235,0,16,6,14,441,100559,1499,506,94,0,48,36068,0.210806
8,1001020500,Alabama,Autauga County,8,4,3,0,6,1,0,0,0,5,54,11,29,8,0,1,3,1,1,0,1,6,0,0,0,0,0,1,383,5,1,0,0,0,36067,0.000804
9,1001020600,Alabama,Autauga County,3620,1765,1855,108,2559,908,0,0,0,2687,44549,16126,27038,4427,492,731,912,890,1049,293,474,3004,173,0,25,126,289,763,49376,3040,510,68,0,220,36067,1.000000


In [ ]:
min(merge['zip']), max(merge['zip'])

(501, 99929)

## Map Zip Codes to City

In [ ]:
zip_to_city.columns

Index(['zip', 'lat', 'lng', 'City', 'State_ID', 'State', 'zcta', 'parent_zcta',
       'population', 'density', 'County_Fips', 'County', 'county_weights',
       'county_names_all', 'county_fips_all', 'imprecise', 'military',
       'timezone'],
      dtype='object')

In [ ]:
zip_to_city = zip_to_city[['zip', 'City', 'State', 'State_ID']]

In [ ]:
merge = merge.drop(columns = ['res_ratio'])

In [ ]:
merge['zip'].nunique(), zip_to_city['zip'].nunique()

(38885, 32914)

In [ ]:
merge[~merge['zip'].isin(zip_to_city['zip'])]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip
1,1001020200,Alabama,Autauga County,61,32,28,0,25,33,0,0,0,46,1166,107,536,69,13,21,18,15,14,3,9,55,5,0,0,0,0,13,522,46,9,5,0,2,36008
7,1001020500,Alabama,Autauga County,2100,1065,1035,18,1627,344,0,64,0,1523,14364,3040,7779,2256,256,375,1024,289,430,73,281,1825,235,0,16,6,14,441,100559,1499,506,94,0,48,36068
48,1003010701,Alabama,Baldwin County,1097,539,558,47,1010,15,3,7,0,808,10994,1273,5450,448,38,8,614,91,233,74,82,1007,30,0,9,4,44,289,40914,838,214,40,4,25,36577
81,1003011201,Alabama,Baldwin County,1353,699,654,29,1181,133,0,7,0,924,16039,1758,11322,2524,286,491,578,265,281,98,128,1063,170,0,21,5,91,328,25250,1063,155,133,0,33,36533
94,1003011407,Alabama,Baldwin County,2941,1423,1518,73,2809,0,0,58,0,2725,25949,6699,20475,3771,543,402,879,758,899,279,123,2191,399,0,0,111,234,764,66101,2447,211,281,0,229,36547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166805,56025000200,Wyoming,Natrona County,1814,1135,678,241,1164,186,18,27,0,1432,14600,2599,8400,905,391,449,395,378,315,375,346,1498,174,12,92,36,1,263,41320,1656,117,30,10,79,82602
166813,56025000502,Wyoming,Natrona County,1661,925,735,295,1296,9,0,27,0,1186,35045,6209,15447,1895,230,365,514,380,378,222,162,1456,139,0,0,11,52,227,26069,1300,292,68,0,159,82605
166910,56037970902,Wyoming,Sweetwater County,2346,1228,1117,433,1599,96,23,4,0,1544,40432,4380,14310,1209,269,417,792,276,562,468,248,1808,340,86,74,16,16,405,61584,1874,370,98,0,58,82902
166930,56039967702,Wyoming,Teton County,4099,2435,1663,384,3463,61,0,171,0,3098,54071,9566,34521,6273,356,0,1454,1257,786,429,171,2750,294,278,323,258,192,549,152038,3360,475,262,0,44,83002
